In [1]:
import os

def count_images(directory):
    exts = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    return sum(
        1
        for entry in os.listdir(directory)
        if os.path.isfile(os.path.join(directory, entry)) 
           and os.path.splitext(entry)[1].lower() in exts
    )

images_dir     = "Images"
stego_images_dir = "stego"

total_images        = count_images(images_dir)
total_stego_images  = count_images(stego_images_dir)

print(f"Total images in '{images_dir}': {total_images}")
print(f"Total images in '{stego_images_dir}': {total_stego_images}")


Total images in 'Images': 8984
Total images in 'stego': 1000


In [7]:
import random
from pathlib import Path
import random
from pathlib import Path
from PIL import Image
import numpy as np
def lsb_extract(stego_path):
    """
    Extract a UTF-8 text message embedded with lsb_embed() above.
    Reads the 32-bit length header, then that many bits from the LSB of the blue channel.
    """
    img    = Image.open(stego_path).convert("RGB")
    pixels = np.array(img, dtype=np.uint8)
    h, w, _ = pixels.shape
    blue    = pixels[:, :, 2].flatten()

    # Read 32-bit header
    header_bits = "".join(str(blue[i] & 1) for i in range(32))
    bit_len     = int(header_bits, 2)

    # Extract payload bits
    bits = "".join(str(blue[i] & 1) for i in range(32, 32 + bit_len))

    # Reconstruct bytes
    data = bytearray()
    for i in range(0, len(bits), 8):
        byte = bits[i : i+8]
        data.append(int(byte, 2))

    return data.decode("utf-8", errors="ignore")

# ──────────────────────────────────────────────────────────────────────────────
# ASSUMPTIONS:
#  • lsb_extract() and iwt_extract() are defined/imported above
# ──────────────────────────────────────────────────────────────────────────────

STEG_DIR = Path("stego")  # or "stego_images" if that's your folder

# 1) List all files in the stego directory
all_stego = [p for p in STEG_DIR.iterdir() if p.is_file()]

if not all_stego:
    print(f"No images found in '{STEG_DIR}'.")
    exit()

# 2) Pick one at random
chosen = random.choice(all_stego)
print("Selected:", chosen.name)

# 3) Extract based on filename prefix
stem = chosen.stem.lower()
if stem.startswith("lsb_"):
    secret = lsb_extract(str(chosen))
    method = "LSB"
elif stem.startswith("iwt_"):
    secret = iwt_extract(str(chosen))
    method = "IWT"
else:
    secret = None
    method = "None"

# 4) Display result
if secret:
    print(f"Method: {method}\nExtracted secret:\n{secret}")
else:
    print(f"Method: {method}\nNo embedded secret found or unrecognized filename prefix.")


Selected: lsb_949.jpg
Method: LSB
Extracted secret:
[SR] I didn’t pull the trigger, but I drove her away.


In [6]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

###############################################################################
# 1) INTEGER HAAR 1D (FORWARD & INVERSE)
###############################################################################
def int_haar_1d_forward(signal):
    """
    Integer 1D Haar Transform (lifting-based) on a 1D array of even length.
    Produces approx[] and detail[] as integers.
    """
    n = len(signal)
    half = n // 2
    approx = np.zeros(half, dtype=int)
    detail = np.zeros(half, dtype=int)

    for i in range(half):
        s0 = signal[2*i]
        s1 = signal[2*i+1]
        d = s1 - s0
        a = s0 + (d // 2)
        detail[i] = d
        approx[i] = a

    return approx, detail

def int_haar_1d_inverse(approx, detail):
    """
    Inverse Integer 1D Haar Transform (lifting-based).
    """
    half = len(approx)
    n = half * 2
    out = np.zeros(n, dtype=int)
    for i in range(half):
        a = approx[i]
        d = detail[i]
        s0 = a - (d // 2)
        s1 = d + s0
        out[2*i] = s0
        out[2*i+1] = s1
    return out

###############################################################################
# 2) INTEGER HAAR 2D (FORWARD & INVERSE)
###############################################################################
def int_haar_2d_forward(img_array):
    """
    Single-level 2D Integer Haar Transform on a 2D array (H x W).
    Returns (LL, LH, HL, HH), all int arrays.
    Assumes even width, even height.
    """
    h, w = img_array.shape

    # 1) Transform each row
    row_approx = np.zeros_like(img_array, dtype=int)
    row_detail = np.zeros_like(img_array, dtype=int)
    for r in range(h):
        approx, detail = int_haar_1d_forward(img_array[r, :])
        row_approx[r, 0:(w//2)] = approx
        row_detail[r, 0:(w//2)] = detail

    # 2) Transform columns of row_approx => (LL, HL)
    #    Transform columns of row_detail => (LH, HH)
    LL = np.zeros((h//2, w//2), dtype=int)
    HL = np.zeros((h//2, w//2), dtype=int)
    LH = np.zeros((h//2, w//2), dtype=int)
    HH = np.zeros((h//2, w//2), dtype=int)

    for c in range(w//2):
        col_data = row_approx[:, c]
        a_col, d_col = int_haar_1d_forward(col_data)
        for rr in range(h//2):
            LL[rr, c] = a_col[rr]
            HL[rr, c] = d_col[rr]

    for c in range(w//2):
        col_data = row_detail[:, c]
        a_col, d_col = int_haar_1d_forward(col_data)
        for rr in range(h//2):
            LH[rr, c] = a_col[rr]
            HH[rr, c] = d_col[rr]

    return LL, LH, HL, HH

def int_haar_2d_inverse(LL, LH, HL, HH):
    """
    Inverse Single-level 2D Integer Haar Transform.
    """
    h2, w2 = LL.shape
    h = h2 * 2
    w = w2 * 2

    # Rebuild row_approx from LL, HL
    row_approx = np.zeros((h, w2), dtype=int)
    row_detail = np.zeros((h, w2), dtype=int)

    for c in range(w2):
        approx_col = LL[:, c]
        detail_col = HL[:, c]
        col_data = int_haar_1d_inverse(approx_col, detail_col)
        for rr in range(h):
            row_approx[rr, c] = col_data[rr]

    for c in range(w2):
        approx_col = LH[:, c]
        detail_col = HH[:, c]
        col_data = int_haar_1d_inverse(approx_col, detail_col)
        for rr in range(h):
            row_detail[rr, c] = col_data[rr]

    # Now inverse row-based transform
    out_array = np.zeros((h, w), dtype=int)
    for rr in range(h):
        a_row = row_approx[rr, :]
        d_row = row_detail[rr, :]
        row_data = int_haar_1d_inverse(a_row, d_row)
        out_array[rr, :] = row_data

    return out_array

###############################################################################
# 3) EMBED & EXTRACT (LH SUB-BAND)
###############################################################################
def iwt_embed(cover_path, secret_message, stego_path):
    """
    1) Load grayscale image, auto-resize to even dims (round up).
    2) int_haar_2d_forward => (LL, LH, HL, HH)
    3) Embed bits in LH's LSB
    4) int_haar_2d_inverse => stego
    5) Save as PNG
    """
    cover_img = Image.open(cover_path).convert("L")
    w, h = cover_img.size

    # Round up to even dims if needed
    new_w = w if (w % 2 == 0) else (w + 1)
    new_h = h if (h % 2 == 0) else (h + 1)
    if (new_w != w) or (new_h != h):
        cover_img = cover_img.resize((new_w, new_h), Image.LANCZOS)
        print(f"Resized image from {w}x{h} to {new_w}x{new_h} for even dims.")
        w, h = new_w, new_h

    cover_array = np.array(cover_img, dtype=int)

    # Forward IWT
    LL, LH, HL, HH = int_haar_2d_forward(cover_array)

    # Prepare bits
    secret_bytes = secret_message.encode('utf-8')
    secret_bits = ''.join(f'{byte:08b}' for byte in secret_bytes)
    end_marker = '1111111111111110'
    full_bits = secret_bits + end_marker

    lh_flat = LH.flatten()
    if len(full_bits) > len(lh_flat):
        raise ValueError("Not enough capacity in LH sub-band.")

    # Embed
    for i, bit in enumerate(full_bits):
        val = lh_flat[i]
        if bit == '1':
            val |= 1
        else:
            val &= ~1
        lh_flat[i] = val

    LH_modified = lh_flat.reshape(LH.shape)

    # Inverse IWT
    stego_array = int_haar_2d_inverse(LL, LH_modified, HL, HH)
    stego_array = np.clip(stego_array, 0, 255).astype('uint8')

    # Save
    stego_img = Image.fromarray(stego_array, mode='L')
    stego_img.save(stego_path, format="PNG")
    print(f"IWT embedding done. Saved stego image to: {stego_path}")

def iwt_extract(stego_path):
    """
    1) Load stego (grayscale), auto-resize to even dims if needed? (Better not, or you'll mismatch)
    2) int_haar_2d_forward => (LL, LH, HL, HH)
    3) Read bits from LH until marker '1111111111111110'
    """
    stego_img = Image.open(stego_path).convert("L")
    w, h = stego_img.size

    # Because we resized in embed, the stego is definitely even dims
    # but if you want to be safe, do the same approach (round up).
    # In theory, the stego is EXACT dims from embed time, so we can skip resizing.
    if (w % 2 != 0) or (h % 2 != 0):
        print(f"Warning: stego has odd dims {w}x{h}, forcibly resizing. Might cause mismatch.")
        new_w = w if (w % 2 == 0) else (w + 1)
        new_h = h if (h % 2 == 0) else (h + 1)
        stego_img = stego_img.resize((new_w, new_h), Image.LANCZOS)
        w, h = new_w, new_h

    stego_array = np.array(stego_img, dtype=int)

    LL, LH, HL, HH = int_haar_2d_forward(stego_array)

    lh_flat = LH.flatten()
    marker = '1111111111111110'
    bits_collected = []

    for val in lh_flat:
        bit = val & 1
        bits_collected.append(str(bit))

        if len(bits_collected) >= len(marker):
            recent = ''.join(bits_collected[-len(marker):])
            if recent == marker:
                all_bits = ''.join(bits_collected[:-len(marker)])
                secret_bytes = []
                for i in range(0, len(all_bits), 8):
                    chunk = all_bits[i:i+8]
                    if len(chunk) < 8:
                        break
                    secret_bytes.append(int(chunk, 2))
                return bytes(secret_bytes).decode('utf-8', errors='ignore')
    return ""


In [12]:
import random
from pathlib import Path

# ──────────────────────────────────────────────────────────────────────────────
# ASSUMPTIONS:
#  • iwt_extract() is defined/imported above
# ──────────────────────────────────────────────────────────────────────────────

STEGO_IWT_DIR = Path("stego_iwt")  # your IWT‐output folder

# 1) Gather all IWT stego files
all_iwt = [
    p for p in STEGO_IWT_DIR.iterdir()
    if p.is_file() and p.suffix.lower() in {".png", ".jpg", ".jpeg", ".bmp"}
]

if not all_iwt:
    print(f"No images found in '{STEGO_IWT_DIR}'.")
    exit()

# 2) Pick one at random
chosen = random.choice(all_iwt)
print("Selected:", chosen.name)

# 3) Extract its hidden message
try:
    secret = iwt_extract(str(chosen))
    print("\nExtracted IWT secret message:\n", secret)
except Exception as e:
    print(f"Failed to extract from {chosen.name}: {e}")


Selected: iwt_882.jpg

Extracted IWT secret message:
 [LOTR] One does not simply walk into Mordor.


In [13]:
import random
from pathlib import Path
import random
from pathlib import Path
from PIL import Image
import numpy as np
def lsb_extract(stego_path):
    """
    Extract a UTF-8 text message embedded with lsb_embed() above.
    Reads the 32-bit length header, then that many bits from the LSB of the blue channel.
    """
    img    = Image.open(stego_path).convert("RGB")
    pixels = np.array(img, dtype=np.uint8)
    h, w, _ = pixels.shape
    blue    = pixels[:, :, 2].flatten()

    # Read 32-bit header
    header_bits = "".join(str(blue[i] & 1) for i in range(32))
    bit_len     = int(header_bits, 2)

    # Extract payload bits
    bits = "".join(str(blue[i] & 1) for i in range(32, 32 + bit_len))

    # Reconstruct bytes
    data = bytearray()
    for i in range(0, len(bits), 8):
        byte = bits[i : i+8]
        data.append(int(byte, 2))

    return data.decode("utf-8", errors="ignore")

# ──────────────────────────────────────────────────────────────────────────────
# ASSUMPTIONS:
#  • lsb_extract() and iwt_extract() are defined/imported above
# ──────────────────────────────────────────────────────────────────────────────

STEG_DIR = Path("stego")  # or "stego_images" if that's your folder

# 1) List all files in the stego directory
all_stego = [p for p in STEG_DIR.iterdir() if p.is_file()]

if not all_stego:
    print(f"No images found in '{STEG_DIR}'.")
    exit()

# 2) Pick one at random
chosen = random.choice(all_stego)
print("Selected:", chosen.name)

# 3) Extract based on filename prefix
stem = chosen.stem.lower()
if stem.startswith("lsb_"):
    secret = lsb_extract(str(chosen))
    method = "LSB"
elif stem.startswith("iwt_"):
    secret = iwt_extract(str(chosen))
    method = "IWT"
else:
    secret = None
    method = "None"

# 4) Display result
if secret:
    print(f"Method: {method}\nExtracted secret:\n{secret}")
else:
    print(f"Method: {method}\nNo embedded secret found or unrecognized filename prefix.")


Selected: lsb_706.jpg
Method: LSB
Extracted secret:
[AV] On your left.
